In [214]:
import mikeio
import mikeio1d
from mikeio1d.res1d import Res1D
import pandas as pd
import numpy as np
import plotly
import plotly.graph_objects as go
from plotly.subplots import make_subplots
# import copy


In [216]:
master_list = []
model_area = 'FSA'
model = 'FSA_Base_2020pop_V044.mdb'
output_folder = r'J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Allways_Latest_Master_Model_Simulations\Plots'
result_folder = r"J:\SEWER_AREA_MODELS\FSA\03_SIMULATION_WORK\Allways_Latest_Master_Model_Simulations\Model"
res_list = []
res_list.append(['2yr 24h No Climate Change','FSA_WWF_EX-2yr-24hr-AES_2020pop_Base.res1d',1])
res_list.append(['5yr 24h No Climate Change','FSA_WWF_EX-5yr-24hr-AES_2020pop_Base.res1d',1])
res_list.append(['10yr 24h No Climate Change','FSA_WWF_EX-10yr-24hr-AES_2020pop_Base.res1d',1])
res_list.append(['Dry Weather Flow','FSA_DWF_2021-07-23_4d_2020pop_Base.res1d',0])
res_list.append(['Jan 31 2020 Event','FSA_WWF_2020-01-29_6d_2020pop_Base.res1d',0])
res_list.append(['Nov 15 2021 Event','FSA_WWF_2021-11-12_5d_2020pop_Base.res1d',0])
master_list.append([model_area,model,result_folder,output_folder,res_list])

In [217]:

for m in master_list:
    model = m[0]
    output_folder = m[1]
    result_folder = m[2]
    output_folder = m[3]
    result_list = m[4]
#     result_list = copy.deepcopy(m[4]) #Make independent copy of list of lists
    nodes_df = pd.read_csv(result_folder + '\\Connection_Stats_Nodes.csv',dtype={'MUID': str, 'Asset Name': str})
    nodes_df['Asset Name'].fillna('',inplace=True)
    lines_df =  pd.read_csv(result_folder + '\\Connection_Stats_Lines.csv',dtype={'MUID': str, 'Acronym': str, 'From Node': str, 'To Node': str})     
    lines_df.Acronym.fillna('',inplace=True)
    
    #Ensure that result_list has design storms next to each other.
    r1 = []
    r2 = []
    for r in result_list:
        if r[2] == 1:
            r1.append(r)
        else:
            r2.append(r)
            
    last_design_index = len(r1) - 1
    result_list = r1 + r2

    for r in result_list:
        header = r[0]
        file = r[1]
        res1d = Res1D(result_folder + '\\' + file)
        r.append(res1d)
        res1d = Res1D(result_folder + '\\' + file[:-6] + '.ADDOUT.res1d')
        r.append(res1d)        
#         for index, row in nodes_df[:3].iterrows():
#             print(row['MUID'])
#             for column in nodes_df.columns[3:]:
#                 print(column)

In [218]:
result_list
# m[4]

[['2yr 24h No Climate Change',
  'FSA_WWF_EX-2yr-24hr-AES_2020pop_Base.res1d',
  1,
 ['5yr 24h No Climate Change',
  'FSA_WWF_EX-5yr-24hr-AES_2020pop_Base.res1d',
  1,
 ['10yr 24h No Climate Change',
  'FSA_WWF_EX-10yr-24hr-AES_2020pop_Base.res1d',
  1,
 ['Dry Weather Flow',
  'FSA_DWF_2021-07-23_4d_2020pop_Base.res1d',
  0,
 ['Jan 31 2020 Event',
  'FSA_WWF_2020-01-29_6d_2020pop_Base.res1d',
  0,
 ['Nov 15 2021 Event',
  'FSA_WWF_2021-11-12_5d_2020pop_Base.res1d',
  0,

In [232]:
colors = ['orchid','lightsalmon','slategrey','purple','green','orange','grey','brown']
flood_types = ['WaterFlowRateAboveGround','WaterSpillDischarge']

class Element_Types:
    def __init__(self,el_type,df,quantity,axis_title,decimals):
        self.el_type = el_type
        self.df = df
        self.quantity = quantity
        self.axis_title = axis_title
        self.decimals = decimals
        
Node = Element_Types('Node',nodes_df,'Water Level','Water Level (m)',2)
Line = Element_Types('Link',lines_df,'Discharge','Discharge (L/s)',1)
        
ETs = [Node,Line]
        


In [239]:
for m in master_list:
    
    result_folder = m[2]
    coordinate_df = pd.read_csv(result_folder + '\\Google_Coordinates.csv',dtype={'MUID': str}).set_index('MUID')
    
    for ET in ETs:

        for index, row in ET.df.iterrows():
            
            plots = []
            muid = row['MUID']
            
#             if muid == '1000107636':
            if 1==1: #Placeholder if no condition
            
                stats = []
                flood_stats = []

                if ET.el_type == 'Node':
                    upstream_df = lines_df[lines_df.ToNode==muid]
                    upstream_lines = list(upstream_df.MUID)
                    upstream_layers = list(upstream_df.Layer)
                    upstreams = zip(upstream_lines,upstream_layers)

                    downstream_df = lines_df[lines_df.FromNode==muid]
                    downstream_lines = list(downstream_df.MUID)
                    downstream_layers = list(downstream_df.Layer)
                    downstreams = zip(downstream_lines,downstream_layers)
                    map_node = muid
                    element_type = 'Node'
                    asset_name = row['Asset Name'].strip()
                    cover_type = row['Cover Type']
                    if row['Cover Type'] == 2:
                        cover_type = 'Sealed '
                    else:
                        cover_type = 'Unsealed '
                else:
                    from_node = row['FromNode']
                    to_node = row['ToNode']
                    layer = row['Layer']
                    fig = go.Figure()
                    if layer == 'Link':
                        muid_adj = muid
                    else:
                        muid_adj = layer + ':' + muid 
                    map_node = from_node
                    element_type = layer
                    asset_name = row['Acronym'].strip()
                    cover_type = ''

                with open(output_folder + "\\" + model + "_" + ET.quantity.replace(' ','') + "_" + element_type + "_" + muid + ".html", 'w') as f:
                    f.write('<link rel="stylesheet" href="style.css">\n')
                    f.write('<div class="sidenav">\n')

                    f.write("<h2>Links to Connected Elements</h2>\n")

                    if ET.el_type == 'Node':
                        for upstream in upstreams:
                            path = output_folder + "\\" + model + "_Discharge_" + upstream[1] + "_" + upstream[0] + ".html"
                            f.write('<a href="' + path + '">Upstream ' + upstream[1] + ' ' + upstream[0] + '</a>\n')
                            f.write("<br>")
                        for downstream in downstreams:
                            path = output_folder + "\\" + model + "_Discharge_" + downstream[1] + "_" + downstream[0] + ".html"
                            f.write('<a href="' + path + '">Downstream ' + downstream[1] + ' ' + downstream[0] + '</a>\n')
                            f.write("<br>")
                    else:  
                        path = output_folder + "\\" + model + "_WaterLevel_Node_" + from_node + ".html"
                        f.write('<a href="' + path + '">Upstream Node ' + from_node + '</a>\n')
                        f.write("<br>")

                        try:
                            path = output_folder + "\\" + model + "_WaterLevel_Node_" + to_node + ".html"
                            f.write('<a href="' + path + '">Downstream Node ' + to_node + '</a>\n')
                        except:
                            f.write('<p>There is no downstream node.</p>\n')
                        f.write("<br>")


                    for i, r in enumerate(result_list):

                        if r[2] == 0 or i == 0:    
                            if ET.el_type == 'Node':
                                fig = make_subplots(specs=[[{"secondary_y": True}]])
                            else:
                                fig = go.Figure()

                            color_index = 0
                            if ET.el_type == 'Node':
                                time_bookends = [r[3].time_index[0],r[3].time_index[-1]]
                                for column in nodes_df.columns[3:]:
                                    if not np.isnan(row[column]):
                                        fig.add_trace(go.Scatter(x=time_bookends, 
                                                         y = [row[column],row[column]], 
                                                         mode='lines',name=column))
                        flood_values = None
                        if ET.el_type == 'Node':
                            values = list(r[3].query.GetNodeValues(muid, "WaterLevel"))                        
                            for j, flood_type in enumerate(flood_types):
                                flood_values = r[4].query.GetNodeValues(muid,flood_type)
                                if cover_type == 'Sealed ':
                                    flood_stats.append([0,0])
                                if flood_values != None:
                                    flood_values = list(flood_values)
                                    flood_values = [k * 1000 for k in flood_values]
                                    flood_stats.append([round(max(flood_values),1),round(sum(flood_values)*300/1000,0)])
#                                 else:
#                                     flood_stats.append(round(0,1))
                        else:
                            values = list(r[3].query.GetReachEndValues(muid_adj, "Discharge"))
                            values = [k * 1000 for k in values]
                        stats.append([r[0],round(max(values),ET.decimals)])
                        fig.add_trace(go.Scatter(x=r[3].time_index, 
                                                     y = values, 
                                                     mode='lines',line_color=colors[color_index],name=r[0]))
                        if flood_values != None:
                            fig.add_trace(go.Scatter(x=r[3].time_index, 
                                                     y = flood_values, 
                                                     mode='lines',line_color=colors[color_index],
                                                     line_dash='dash',name=r[0] + ' Spilling'),secondary_y=True)

                        color_index += 1


                        if r[2] == 0 or i == last_design_index:
                            fig.update_layout(
                                autosize=False,
                                width = 1362,
                                height=500,
                                margin=dict(
                                    l=50,
                                    r=50,
                                    b=25,
                                    t=35,
                                    pad=4
                                    ),
                                yaxis_title=ET.axis_title
                                )
                            if ET.el_type == 'Node':                            
                                fig['layout']['yaxis2']['title']='Manhole Spilling (L/s)'
                                fig.update_layout(yaxis2_rangemode="tozero")

                            if r[2] == 0:
                                plot_header = "<h2>" + r[0] + "</h2>\n"
                            else:
                                plot_header = "<h2>Design Storms</h2>\n"
                            plots.append([plot_header,fig])
                    #Statistics
                    f.write("<h2>Statistics</h2>\n")                               
                    f.write("<table>\n")
                    f.write("  <tr>\n")
                    f.write("    <th>Description</th>\n")
                    if ET.el_type == 'Node':
                        f.write("    <th>HGL (m)    </th>\n")
                        f.write("    <th>Spill (L/s)</th>\n")
                        f.write("    <th>Spill (m<sup>3</sup>)</th>\n")
                    else:
                        f.write("    <th>Maximum " + ET.axis_title + "</th>\n")
                    f.write("  </tr>\n")
                    if ET.el_type == 'Node':
                        f.write("  <tr>\n")
                        f.write("    <td>Safe Operating Head</td>\n")
                        if np.isnan(row['Safe Operating Head']):
                            f.write("    <td>None</td>\n")
                        else:
                            f.write("    <td>" + str(round(row['Safe Operating Head'],2)) + "</td>\n")
                        f.write("    <td></td>\n")
                        f.write("    <td></td>\n")
                        f.write("  </tr>\n")                    
                    for j, stat in enumerate(stats):
                        f.write("  <tr>\n")
                        f.write("    <td>" + stat[0] + "</td>\n")  
                        f.write("    <td>" + str(stat[1]) + "</td>\n")
                        if ET.el_type == 'Node':
                            f.write("    <td>" + str(flood_stats[j][0]) + "</td>\n")
                            f.write("    <td>" + str(flood_stats[j][1]) + "</td>\n")
                        f.write("  </tr>\n")                   
                    f.write("</table>\n")

                    f.write("<h2>Map</h2>\n")
                    map_string = '<img src="Maps\\' + map_node + '.jpg" alt="' + map_node + '">\n'
                    f.write(map_string + "\n")
                    f.write("<p></p>\n")
                    google_map_string = '<a href="https://maps.google.com/?q='
                    google_map_string += str(coordinate_df.loc[map_node,'Y']) + ', '
                    google_map_string += str(coordinate_df.loc[map_node,'X']) + '" '
                    google_map_string += 'target="_blank">View in Google Maps (Pipes not shown)</a>'
                    f.write(google_map_string + "\n")

                    f.write('</div>')

                    f.write('<div class="main">\n')

                    asset_name_str = ''
                    if asset_name != '':
                        asset_name_str = ' (' + asset_name + ')'

                    f.write("<h1>" + ET.quantity + " in " + cover_type + muid + asset_name_str + "</h1>\n")

                    for plot in plots:
                        f.write(plot[0])
                        f.write(plot[1].to_html(full_html=False, include_plotlyjs='cdn'))
                    f.write('</div>')
                f.close()


IndexError: list index out of range

In [236]:
muid

'1_1129_N'

In [7]:
map_node = '4785'
strr = '<img src="Maps\\' + map_node + '.jpg" alt="' + map_node + '">'
print(strr)  

<img src="Maps\4785.jpg" alt="4785">


In [164]:
for j, flood_type in enumerate(flood_types):
    flood_values = r[4].query.GetNodeValues('1000107636',flood_type)
    if flood_values != None:
        print("Flood in " + r[0])
        flood_values = list(flood_values)
        flood_values = [k * 1000 for k in flood_values]

In [169]:
result_list

[['2yr 24h No Climate Change',
  'FSA_WWF_EX-2yr-24hr-AES_2020pop_Base.res1d',
  1,
 ['5yr 24h No Climate Change',
  'FSA_WWF_EX-5yr-24hr-AES_2020pop_Base.res1d',
  1,
 ['10yr 24h No Climate Change',
  'FSA_WWF_EX-10yr-24hr-AES_2020pop_Base.res1d',
  1,
 ['Dry Weather Flow',
  'FSA_DWF_2021-07-23_4d_2020pop_Base.res1d',
  0,
 ['Jan 31 2020 Event',
  'FSA_WWF_2020-01-29_6d_2020pop_Base.res1d',
  0,
 ['Nov 15 2021 Event',
  'FSA_WWF_2021-11-12_5d_2020pop_Base.res1d',
  0,

In [167]:
r[4].query.GetNodeValues('1000107636','WaterSpillDischarge')

In [107]:
r[2]

0

In [73]:
str(row['Acronym'])

'nan'

In [77]:
lines_df.Acronym.fillna('',inplace=True) 
lines_df

,MUID,Acronym,FromNode,ToNode,Layer
0,1000082441,,1000107574,1000107573,Link
1,1000082444,,1000107575,1000107574,Link
2,1000082453,,1000107576,1000107577,Link
3,1000082454,,Node_7738,Node_7739,Link
4,1000082455,,1000107577,1000107575,Link
...,...,...,...,...,...
7415,WhiteRock2a-2ndLevel,White Rock PS,35445,24127,Pump
7416,WhiteRock3a-3rdLevel,White Rock PS,35445,24127,Pump
7417,WhiteRockMunPump2,White Rock Mun PS,4095,4094,Pump
7418,Glenbrook Chamber Connection Valve,,22662,89518,Valve


In [149]:
flood_values

In [150]:
a = None


In [151]:
a